## Utilizing Hierarchical Structure for Audio-Based Music Similarity
#### Christos Plachouras, ISMIR 2021 Late-Break Demo, [Code](https://github.com/chrispla/align-and-compare), [Paper](), [Poster](), [Video]()


This notebook provides code for demonstrating how the method works for comparing two audio files.

### > Library importing

In [ ]:
# for audio processing
import librosa
# for plotting
import librosa.display
import matplotlib.pyplot as plt 
# for matrix processing
import numpy as np
import scipy
# for spectral clustering
import sklearn.cluster

### > Loading audio
Choose paths for the two audio files

In [ ]:
# File paths
path1 = 'enter/path/here'
path2 = 'enter/path/here'

### > Hierarchical structure analysis

Code modified from [Brian McFee - Laplacian Segmentation (MIT License)](https://github.com/bmcfee/lsd_viz)

Brian McFee, Dan Ellis. "Analyzing Song Structure with Spectral Clustering". ISMIR 2014.


In [ ]:
# Constant-Q Transform parameters
BINS_PER_OCTAVE = 12 * 3
N_OCTAVES = 7

# Spectral clustering parameters (spectral clustering component translates to number of segment types)
kmin = 2  # needs to be >=2
kmax = 7

#### >> Compute Laplacian
First, we compute the Constant-Q Transform of the audio file and beat-synchronize it. Then, we compute an affinity matrix encoding the self-similarity of each frame of the CQT, and combine it with a sequence matrix encoding the local consistency of MFCCs. Finally, we compute the Laplacian to shift it to its matrix representation.

In [ ]:
def compute_laplacian(path, BINS_PER_OCTAVE, N_OCTAVES):
    """
    Compute the Laplacian matrix from an audio file using
    its Constant-Q Transform

    Input
    -----
    path: filepath (str)

    Output
    -----
    C: Constant-Q Transform (np.array)
    L: Laplacian matrix (np.array)
    """

    # load audio
    y, sr = librosa.load(path, sr=22050, mono=True)

    # Compute Constant-Q Transform in dB
    C = librosa.amplitude_to_db(np.abs(librosa.cqt(y=y,
                                                   sr=sr,
                                                   bins_per_octave=BINS_PER_OCTAVE,
                                                   n_bins=N_OCTAVES * BINS_PER_OCTAVE)),
                                                   ref=np.max)

    # beat tracking
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr, trim=False)
    # get beat times in seconds
    beat_times = librosa.frames_to_time(librosa.util.fix_frames(beats, x_min=0, x_max=C.shape[1]), sr=sr)

    # beat synchronize the CQT
    Csync = librosa.util.sync(C, beats, aggregate=np.median)

    # stack 4 consecutive frames
    Cstack = librosa.feature.stack_memory(Csync, 4)

    # compute weighted recurrence matrix
    R = librosa.segment.recurrence_matrix(Cstack, width=3, mode='affinity', sym=True)

    # enhance diagonals with a median filter
    df = librosa.segment.timelag_filter(scipy.ndimage.median_filter)
    Rf = df(R, size=(1, 7))
    Rf = librosa.segment.path_enhance(Rf, 15)

    # compute MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    # beat synchronize them
    Msync = librosa.util.sync(mfcc, beats)

    # build the MFCC sequence matrix
    path_distance = np.sum(np.diff(Msync, axis=1)**2, axis=0)
    sigma = np.median(path_distance)
    path_sim = np.exp(-path_distance / sigma)
    R_path = np.diag(path_sim, k=1) + np.diag(path_sim, k=-1)

    # get the balanced combination of the MFCC sequence matric and the CQT
    deg_path = np.sum(R_path, axis=1)
    deg_rec = np.sum(Rf, axis=1)
    mu = deg_path.dot(deg_path + deg_rec) / np.sum((deg_path + deg_rec)**2)
    A = mu * Rf + (1 - mu) * R_path

    # compute the normalized Laplacian
    L = scipy.sparse.csgraph.laplacian(A, normed=True)

    return C, L

#### >> Segment and cluster

We eigendecompose the Laplacian and make sets of its first integer k in [kmin, kmax) eigenvectors. We then perform spectral clustering on each set using the respective number k of clusters. We also compute the Euclidean self-distance of the eigenvectors across time for each set to create "approximations" of the Laplacian for visualization purposes.

In [ ]:
def segment(L, kmin, kmax):
    """
    Decompose Laplacian and make sets of its first integer k in [kmin, kmax] eigenvectors.
    For each set, compute its Euclidean self distance over time, and do spectral clustering.

    Input
    -----
    L: Laplacian matrix (np.array)
    kmin: first eigenvector index
    kmax: last eigenvector index

    Output
    -----
    distances: self distance matrix of each set of first eigenvectors (np.array, shape=(kmax-kmin, beats, beats))
    segments: estimated structural segments for each set of first eigenvectors (np.array, shape=(kmax-kmin, beats))
    """

    # eigendecomposition
    evals, evecs = scipy.linalg.eigh(L)

    # eigenvector filtering
    evecs = scipy.ndimage.median_filter(evecs, size=(9, 1))

    # normalization
    Cnorm = np.cumsum(evecs**2, axis=1)**0.5

    # initialize sets
    distances = []
    segments = []

    for k in range(kmin, kmax):
        # create set using first k normalized eigenvectors
        Xs = evecs[:, :k] / Cnorm[:, k-1:k]

        # get self distance over time of the set
        distances.append(scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(Xs, metric='euclidean')))

        # cluster each set k using k clusters to get k segment types
        KM = sklearn.cluster.KMeans(n_clusters=k, n_init=50, max_iter=500)
        segments.append(KM.fit_predict(Xs))

    return np.asarray(distances), np.asarray(segments)

#### >> Run and plot

In [ ]:
# Compute Laplacian using the Constant-Q Transform
C1, L1 = compute_laplacian(path1, BINS_PER_OCTAVE, N_OCTAVES)
C2, L2 = compute_laplacian(path2, BINS_PER_OCTAVE, N_OCTAVES)

# Compute Laplacian approximations and hierarchical structural analysis 
d1, s1 = segment(L1, kmin, kmax)
d2, s2 = segment(L2, kmin, kmax)

In [ ]:
# get relative scale of beat vectors for ploting
cqt_scale1 = min(1, C1.shape[1]/C2.shape[1])
cqt_scale2 = min(1, C2.shape[1]/C1.shape[1])
seg_scale1 = min(1, s1.shape[1]/s2.shape[1])
seg_scale2 = min(1, s2.shape[1]/s1.shape[1])

# plot Constant-Q Transform
for path, C, scale in zip([path1, path2], [C1, C2], [scale1, scale2]):
    fig1, ax1 = plt.subplots(1, figsize=(15*scale, 5))
    librosa.display.specshow(C, y_axis='cqt_hz', sr=22050,
                                bins_per_octave=BINS_PER_OCTAVE,
                                x_axis='s', ax=ax1)
    ax1.set_title(path+" - Constant-Q Transform")

# plot Laplacian approximations
for path, d in zip([path1, path2], [d1, d2]):
    fig2, ax2 = plt.subplots(1, kmax-kmin, figsize=(15, 3), sharey='all')
    plt.set_cmap('viridis')
    for k in range(kmax-kmin):
        ax2[k].imshow(d[k], origin='lower', cmap='viridis', interpolation='none')
        ax2[k].set_title("k = " + str(kmin+k))
    fig2.suptitle(path+" - Approximations of Laplacian")
    ax2[0].set(xlabel='beats', ylabel='beats')

# plot hierarchical structure
y_ticks = np.arange(0, kmax-kmin)
y_tick_labels = np.arange(kmin, kmax).astype(str)
for path, s, scale in zip([path1, path2], [s1, s2], [seg_scale1, seg_scale2]):
    fig3, ax3 = plt.subplots(1, 1, figsize=(15*scale, 10))
    ax3.imshow(s, interpolation='none', aspect=20)
    ax3.set_title(path+' - Hierarchical Structure')
    ax3.set(xlabel='beats', ylabel='k')
    ax3.set(yticks = y_ticks, yticklabels = y_tick_labels)